In [1]:
import numpy as np 
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import glob
from tqdm import tqdm
from collections import OrderedDict, defaultdict
from IPython.display import clear_output
import time
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [3]:
class LSTM(nn.Module):
    
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num, seq_len):
        super(LSTM,self).__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim,hidden_dim,layer_num,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        self.bn = nn.BatchNorm1d(seq_len)
        
    def forward(self,inputs):
        x = self.bn(inputs)
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [4]:
n_hidden = 128
n_joints = 25*2
n_categories = 2
n_layer = 3
seq_len = 30
model = LSTM(n_joints,n_hidden,n_categories,n_layer, seq_len).to(device)


model.load_state_dict(torch.load('rnn_train_1_dec7.pth'))
model.eval()

LSTM(
  (lstm): LSTM(50, 128, num_layers=3, batch_first=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
  (bn): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [5]:
def extract_imgs(video_fn, skip_frames=1):
    video=cv2.VideoCapture(video_fn)
    imgs=[]
    frame_count=0
    while(video.isOpened()):
        ret, frame = video.read()
        if ret==False: break
        if frame_count%skip_frames==0:
            imgs.append(frame)
        frame_count+=1
    video.release() 
    return imgs

In [6]:
pose = mp.solutions.pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
def extract_keypoints(image_rgb): 
    try:
        results = pose.process(image_rgb)
        landmarks=results.pose_landmarks.landmark
    except Exception as e:
        # print('Error file=', fn)
        # print('Error=', e)
        return None
    xys=[]
    for landmark in landmarks:
        xys.append([landmark.x, landmark.y])
    xys=np.array(xys)
    return xys

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [7]:
dir="/home/ns/data_activity/clips_eating/"
fn=dir+"Dain Eating 1_1.mp4"

In [10]:
imgs=extract_imgs(fn, skip_frames=1)
len(imgs)

52

In [12]:
kpss=[]
for img in imgs:
    kps=extract_keypoints(img)
    kpss.append(kps[:25].flatten())

kpss=np.array(kpss)
kpss.shape

(52, 50)

In [14]:
Xt=kpss[:seq_len]
Xt=torch.tensor(Xt).unsqueeze(0).float().to(device)
Xt.shape

torch.Size([1, 30, 50])

In [17]:
logits=model(Xt).detach().cpu().numpy()
po=np.argmax(logits)
po

0